# **Chapter 13: Loading and Preprocessing Data with TensorFlow**

## **1. Pendahuluan**

Sejauh ini, kita sering bekerja dengan dataset yang cukup kecil untuk dimuat seluruhnya ke dalam memori (RAM). Namun, sistem Deep Learning modern sering kali dilatih menggunakan dataset yang sangat besar yang tidak muat dalam RAM. Mengelola dataset besar dan melakukan preprocessing secara efisien bisa menjadi hal yang rumit, tetapi TensorFlow menyederhanakannya melalui **Data API**.

Data API memungkinkan kita untuk membuat objek dataset, memberi tahu dari mana data diambil, dan bagaimana cara mentransformasikannya. TensorFlow menangani detail implementasi seperti *multithreading*, *queuing*, *batching*, dan *prefetching* secara otomatis. Selain itu, Data API bekerja mulus dengan `tf.keras`.

Dalam bab ini, kita akan membahas:
* **Data API**: Membangun *input pipeline* yang efisien.
* **TFRecord Format**: Format biner fleksibel dan efisien untuk data besar.
* **Preprocessing**: Menggunakan layer preprocessing Keras standar dan kustom (seperti encoding fitur kategorikal dan *embeddings*).
* **TensorFlow Datasets (TFDS)**: Cara mudah memuat dataset standar.

## **2. The Data API**

Konsep inti dari Data API adalah `dataset`, yang merepresentasikan urutan item data. Kita bisa membaca data dari disk, namun untuk mempermudah pemahaman awal, mari kita buat dataset dari memori menggunakan `tf.data.Dataset.from_tensor_slices()`.

In [ ]:
import tensorflow as tf

# Membuat dataset dari tensor sederhana (angka 0 sampai 9)
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)

# Iterasi item dalam dataset
for item in dataset:
    print(item.numpy())

### **Rantai Transformasi (Chaining Transformations)**

Setelah memiliki objek dataset, kita dapat menerapkan berbagai transformasi. Setiap metode transformasi mengembalikan dataset baru, sehingga kita bisa merangkainya (*chaining*).

Operasi umum meliputi:
* `repeat(n)`: Mengulangi dataset sebanyak `n` kali.
* `batch(n)`: Mengelompokkan item ke dalam batch berukuran `n`.
* `map(fn)`: Menerapkan fungsi transformasi pada setiap item.
* `shuffle(buffer_size)`: Mengacak data untuk memastikan independensi instance training.
* `filter(fn)`: Menyaring data berdasarkan kondisi tertentu.
* `take(n)`: Mengambil `n` item pertama saja.

In [ ]:
# Contoh rantai transformasi: Repeat, Batch, dan Map
# 1. Ulangi dataset 3 kali
# 2. Kelompokkan dalam batch berisi 7 item
# 3. Gandakan nilai setiap item (map)

dataset = dataset.repeat(3).batch(7)
dataset = dataset.map(lambda x: x * 2)

for item in dataset:
    print(item.numpy())

**Penjelasan:**
Metode `batch(7)` akan menghasilkan tensor berukuran 7. Jika sisa data di akhir kurang dari 7, batch terakhir akan lebih kecil (kecuali jika `drop_remainder=True` digunakan).

### **Mengacak Data (Shuffling)**

Gradient Descent bekerja paling baik jika data training bersifat *independent and identically distributed* (IID). Metode `shuffle()` menggunakan buffer untuk mengacak data.

Penting untuk menentukan `buffer_size` yang cukup besar agar pengacakan efektif, tetapi tidak melebihi kapasitas RAM.

In [ ]:
# Mengacak dataset dengan buffer size 5 dan random seed 42
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)

for item in dataset:
    print(item.numpy())

### **Interleaving: Membaca dari Banyak File**

Untuk dataset skala besar, data biasanya dipecah ke dalam banyak file. Kita dapat membaca file-file ini secara acak dan menyisipkan (*interleave*) baris-barisnya untuk pengacakan yang lebih baik.

Fungsi `interleave()` memungkinkan kita membaca dari beberapa file sekaligus secara paralel. Berikut adalah contoh fungsi `csv_reader_dataset` yang menggabungkan pembacaan file CSV, preprocessing, shuffling, dan batching.

In [ ]:
def csv_reader_dataset(filepaths, repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    # 1. List file paths dan acak urutannya
    dataset = tf.data.Dataset.list_files(filepaths)
    
    # 2. Interleave: Membaca dari n_readers file sekaligus
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1), # skip header
        cycle_length=n_readers, 
        num_parallel_calls=n_read_threads)
    
    # 3. Preprocessing (map) dan Shuffling
    # Asumsi fungsi 'preprocess' sudah didefinisikan sebelumnya
    # dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    
    # 4. Batching dan Prefetching
    return dataset.batch(batch_size).prefetch(1)

### **Prefetching: Meningkatkan Performa**

Metode `prefetch(1)` di akhir pipeline sangat krusial untuk performa. Ini membuat dataset selalu menyiapkan satu batch di depan sementara GPU sedang melatih batch saat ini.



Dengan *prefetching*, CPU dan GPU bekerja secara paralel, sehingga GPU tidak perlu menunggu data dimuat, yang secara dramatis mempercepat waktu pelatihan.

## **3. Format TFRecord**

TFRecord adalah format pilihan TensorFlow untuk menyimpan data dalam jumlah besar. Ini adalah format biner sederhana yang berisi urutan *binary records*. Format ini sangat efisien untuk dibaca secara sekuensial.

### **Menulis dan Membaca TFRecord**
Kita bisa menggunakan `tf.io.TFRecordWriter` untuk menulis dan `tf.data.TFRecordDataset` untuk membaca.

In [ ]:
# Menulis ke file TFRecord
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

# Membaca dari file TFRecord
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

### **Protocol Buffers (Protobufs)**

Meskipun TFRecord bisa menyimpan string biner apa saja, biasanya ia digunakan untuk menyimpan **Protocol Buffers** (protobuf) yang diserialisasi. Protobuf yang paling umum digunakan adalah `Example`.

Protobuf `Example` merepresentasikan satu instance dalam dataset dan berisi daftar fitur bernama. Berikut cara membuat dan menulis `Example` protobuf:

In [ ]:
from tensorflow.train import BytesList, FloatList, Int64List
from tensorflow.train import Feature, Features, Example

# Membuat objek Example Protobuf
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

# Serialisasi dan tulis ke file
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

Untuk membaca kembali data tersebut, kita perlu mendefinisikan deskripsi fitur dan menggunakan `tf.io.parse_single_example`.

## **4. Preprocessing Fitur Input**

Data mentah sering kali perlu dikonversi menjadi numerik, dinormalisasi, atau di-*encode* sebelum masuk ke jaringan saraf.

### **Encoding Fitur Kategorikal (One-Hot)**
Jika kita memiliki fitur kategorikal dengan sedikit kategori (misal: "NEAR BAY", "INLAND"), kita bisa menggunakan *one-hot encoding*. Kita perlu memetakan setiap kategori ke indeks menggunakan *lookup table*.

In [ ]:
# Contoh Lookup Table untuk kategori
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)

# Inisialisasi Lookup Table dengan OOV (Out-of-Vocabulary) buckets
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

# Menguji lookup dan one-hot encoding
categories = tf.constant(["NEAR BAY", "DESERT", "INLAND", "INLAND"])
cat_indices = table.lookup(categories)
cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)

print("Indices:", cat_indices)
print("One-hot:\n", cat_one_hot)

**Penjelasan:**
Kategori yang tidak dikenal (seperti "DESERT") akan dipetakan ke salah satu *oov buckets* menggunakan *hashing*, untuk menghindari hilangnya informasi sepenuhnya.

### **Embeddings**

Untuk fitur kategorikal dengan banyak kategori (misal: 50+), *one-hot encoding* menjadi tidak efisien. Solusinya adalah menggunakan **Embeddings**.

Embedding adalah vektor padat (*dense vector*) yang dapat dilatih, yang merepresentasikan sebuah kategori. Awalnya diinisialisasi secara acak, namun selama pelatihan, *Gradient Descent* akan mendorong kategori yang mirip agar memiliki representasi vektor yang dekat di ruang embedding.



Keras menyediakan layer `keras.layers.Embedding` untuk menangani ini secara otomatis.

In [ ]:
# Implementasi Layer Embedding di Keras
embedding_dim = 2
embed_layer = tf.keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets, 
                                        output_dim=embedding_dim)

# Mengambil vektor embedding untuk indeks kategori
embeddings = embed_layer(cat_indices)
print("Embeddings:\n", embeddings)

### **Keras Preprocessing Layers**

Keras kini menyediakan layer standar untuk preprocessing yang bisa dimasukkan langsung ke dalam model, seperti:
* `Normalization`: Untuk standarisasi fitur (rata-rata 0, deviasi standar 1).
* `TextVectorization`: Untuk mengubah teks menjadi urutan token atau vektor hitungan kata (Bag-of-Words/TF-IDF).
* `Discretization`: Untuk memotong data kontinu menjadi *bin*.

Layer ini memiliki metode `adapt()` yang memungkinkan layer mempelajari statistik data (seperti *vocabulary* atau *mean/std*) sebelum pelatihan dimulai.

## **5. TensorFlow Datasets (TFDS)**

Proyek TensorFlow Datasets (TFDS) memudahkan kita mengunduh dan memuat dataset standar seperti MNIST, ImageNet, atau dataset teks. TFDS menyediakan dataset yang sudah siap digunakan dengan Data API.

Berikut contoh memuat dataset MNIST menggunakan TFDS:

In [ ]:
import tensorflow_datasets as tfds

# Memuat dataset MNIST (as_supervised=True memberikan tuple (fitur, label))
dataset, info = tfds.load(name="mnist", as_supervised=True, with_info=True)
mnist_train, mnist_test = dataset["train"], dataset["test"]

# Menerapkan transformasi pipeline
mnist_train = mnist_train.shuffle(10000).batch(32).prefetch(1)

print("Dataset MNIST siap digunakan!")

## **6. Kesimpulan**

Bab ini mengajarkan kita bahwa memuat dan memproses data secara efisien adalah kunci dalam Deep Learning skala besar. Kita telah mempelajari:
* **Data API** memungkinkan pembuatan pipeline yang kompleks dengan *chaining* metode seperti `map`, `shuffle`, dan `batch`.
* **Prefetching** dan **Interleaving** memaksimalkan penggunaan GPU dengan memparalelkan proses I/O CPU.
* **TFRecord** adalah format yang disarankan untuk penyimpanan data besar yang efisien menggunakan Protocol Buffers.
* **Preprocessing** seperti *One-Hot Encoding* dan *Embeddings* sangat penting untuk menangani data kategorikal dan teks agar dapat diproses oleh Neural Network.

Dengan alat-alat ini, kita siap menangani dataset yang jauh lebih besar dari kapasitas memori komputer kita.